In [1]:
#!pip install pyarrow
#!pip install fastparquet

In [2]:
from pathlib import Path
import pandas as pd
import glob
import re

import warnings
warnings.filterwarnings('ignore')

from IPython.display import display
pd.options.display.max_columns = None

In [3]:
HOME = Path().resolve().parent.parent
data_dir = str(HOME)
input_dir = data_dir + '/data/backup/'
output_dir = data_dir + '/data/processed/'

### 1. Read all the files into a single dataset

In [4]:
all_files = glob.glob(input_dir + "*.csv.gz")
li = []
regex = re.compile(r'\d+')

print('FILEDATE \tMIN_DATE \tMAX_DATE \tDATASET_SHAPE')

for filename in all_files:
    
    # Read the raw data
    df = pd.read_csv(filename, index_col=None)
    
    # Save filename date for future analysis   
    file_date = re.search(regex, filename).group(0)
    df['file_date'] = file_date
    
    # Print data info for each file
    print('{} \t\t {} \t {} \t {}'.format(file_date, df.last_scraped.min(), df.last_scraped.max(), df.shape))
    
    # Append the data in a list
    li.append(df)

FILEDATE 	MIN_DATE 	MAX_DATE 	DATASET_SHAPE
1901 		 2019-01-14 	 2019-01-15 	 (18033, 107)
1902 		 2019-02-06 	 2019-02-06 	 (17763, 107)
1903 		 2019-03-08 	 2019-03-08 	 (17807, 107)
1904 		 2019-04-10 	 2019-04-10 	 (17899, 107)
1905 		 2019-05-14 	 2019-05-14 	 (18302, 107)
1906 		 2019-06-07 	 2019-06-08 	 (18837, 107)
1907 		 2019-07-10 	 2019-07-12 	 (19833, 107)
1908 		 2019-08-12 	 2019-08-12 	 (20556, 107)
1909 		 2019-09-17 	 2019-10-08 	 (20404, 107)
1910 		 2019-10-16 	 2019-10-16 	 (20147, 107)
1911 		 2019-11-09 	 2019-12-03 	 (20428, 107)
1912 		 2019-12-10 	 2019-12-10 	 (20843, 107)
2001 		 2020-01-10 	 2020-02-01 	 (20708, 107)
2002 		 2020-02-16 	 2020-02-29 	 (20981, 107)
2003 		 2020-03-16 	 94% 	 (21117, 75)
2004 		 2020-04-16 	 93% 	 (20839, 75)
2005 		 2020-05-11 	 92% 	 (20859, 75)
2006 		 2020-06-13 	 94% 	 (20865, 75)
2007 		 2020-07-17 	 96% 	 (20518, 75)
2008 		 2020-08-24 	 95% 	 (20704, 75)
2009 		 2020-09-12 	 94% 	 (20338, 75)
2010 		 2020-10-12 	 2020

The are some trash in the ```last_scraped``` column and it must be cleaned later. Also, we are going to save the file date info for future analysis. 

In [5]:
valid_columns = set(li[0].columns.values)

for df in li:
    current_cols = set(df.columns.values)
    valid_columns.intersection_update(current_cols)
    
print('There are {} valid columns in the dataset.'.format(len(valid_columns)))

There are 73 valid columns in the dataset.


In [6]:
for i in range(len(li)):
    li[i] = li[i][valid_columns]

# Aggregate all the data into a sigle dataset:
df = pd.concat(li, axis=0, ignore_index=True)

### 2. Removing columns pointed by Pol:

In [7]:
# Removed 'bathrooms_text' and 'number_of_reviews_l30d' from cols2drop

cols2drop = ['listing_url', 'scrape_id', 'name', 'description', 'neighborhood_overview', 
             'picture_url', 'host_id', 'host_url', 'host_name', 'host_about', 
             'host_response_time', 'host_response_rate', 'host_acceptance_rate', 
             'host_is_superhost', 'host_thumbnail_url', 'host_picture_url', 
             'host_listings_count', 'host_total_listings_count', 'host_verifications',
             'host_has_profile_pic', 'host_identity_verified', 
             'minimum_nights', 'maximum_nights', 'host_location',
             'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 
             'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 
             'host_since', 'neighbourhood', 'calendar_updated', 'review_scores_communication', 
             'review_scores_location', 'license', 'calculated_host_listings_count',
             'calculated_host_listings_count_entire_homes', 
             'calculated_host_listings_count_private_rooms', 
             'calculated_host_listings_count_shared_rooms', 'host_neighbourhood', 
             'neighbourhood_cleansed', 'review_scores_accuracy', 'review_scores_cleanliness', 
             'review_scores_checkin', 'review_scores_value', 'number_of_reviews_ltm', 
             'first_review', 'last_review', 'instant_bookable']

listings = df.drop(columns=cols2drop).copy()
rows, cols = listings.shape
print('The current dataset has {} rows and {} columns.'.format(rows, cols))

The current dataset has 550766 rows and 24 columns.


In [8]:
rows, cols = listings.shape
print('The final dataset has {} rows and {} columns.'.format(rows, cols))

The final dataset has 550766 rows and 24 columns.


### Save the final, aggregated dataset:

In [9]:
listings.to_csv(output_dir + '/listings_agg.csv.gz', index=False)